In [17]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [18]:
z = pd.read_csv('http://ce.sharif.edu/courses/85-86/1/ce925/assignments/files/assignDir2/churn.txt')
z.head(n=2)

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.7,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.7,1,False.


In [19]:
x = z.iloc[:,1:20].values
y = z.iloc[:,20].values

In [20]:
le = LabelEncoder()
x[:,2] = le.fit_transform(x[:,2])
x[:,3] = le.fit_transform(x[:,3])
x[:,4] = le.fit_transform(x[:,4])
y = le.fit_transform(y)

In [21]:
A = x
B = pd.get_dummies(y)

In [22]:
lr_rate = 0.01
epochs = 100
batch_size = 100
num_input = 19
hid1 = 1000
hid2 = 500
hid3 = 250
hid4 = 100
hid5 = 50
num_class = 2

In [23]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

In [24]:
x_train,x_test,y_train,y_test = train_test_split(A,B,test_size = 0.3)


In [25]:
W = {'h1':tf.Variable(tf.random_normal([num_input,hid1])),
     'h2':tf.Variable(tf.random_normal([hid1,hid2])),
     'h3':tf.Variable(tf.random_normal([hid2,hid3])),
     'h4':tf.Variable(tf.random_normal([hid3,hid4])),
     'h5':tf.Variable(tf.random_normal([hid4,hid5])),
    'out':tf.Variable(tf.random_normal([hid5,num_class]))}

B = {'h1':tf.Variable(tf.random_normal([hid1])),
     'h2':tf.Variable(tf.random_normal([hid2])),
     'h3':tf.Variable(tf.random_normal([hid3])),
     'h4':tf.Variable(tf.random_normal([hid4])),
     'h5':tf.Variable(tf.random_normal([hid5])),
    'out':tf.Variable(tf.random_normal([num_class]))}

In [26]:
def neu_network(X,W,B):
    layer_1 = tf.add(tf.matmul(X,W['h1']),B['h1'])
    L1 = tf.nn.tanh(layer_1)
    
    layer_2 =tf.add(tf.matmul(layer_1,W['h2']),B['h2'])
    L2 = tf.nn.tanh(layer_2)
    
    layer_3 = tf.add(tf.matmul(layer_2,W['h3']),B['h3'])
    L3 = tf.nn.tanh(layer_3)
    
    layer_4 = tf.add(tf.matmul(layer_3,W['h4']),B['h4'])
    L4 = tf.nn.sigmoid(layer_4)
    
    layer_5 = tf.add(tf.matmul(layer_4,W['h5']),B['h5'])
    L5 = tf.nn.sigmoid(layer_5)
    
    layer_out = tf.add(tf.matmul(layer_5,W['out']),B['out'])
    return layer_out 

In [27]:
y = neu_network(X,W,B)
y_output = tf.nn.softmax(y)

In [28]:
loss_fun = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y,labels = Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = lr_rate).minimize(loss_fun)

In [29]:
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction , tf.float32))

In [30]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for epoch in range (epochs):
    sess.run(optimizer , feed_dict = {X:x_train,Y:y_train})
    lf = sess.run(loss_fun,feed_dict = {X:x_train ,Y:y_train})
    Accuracy = sess.run(accuracy ,feed_dict = {X:x_train , Y:y_train})
    #print("Epoch:",epoch,'--','Loss:',lf)

In [31]:
print("test accuracy",sess.run(accuracy,feed_dict = {X:x_test,Y:y_test})*100)

test accuracy 86.10000014305115
